In [1]:
# Import pandas for data manipulation and analysis
import pandas as pd

# Import regular expressions for string manipulation
import re

# Import Counter from collections for counting occurrences
from collections import Counter

# Import matplotlib.pyplot for data visualization
import matplotlib.pyplot as plt

# Import seaborn for advanced statistical data visualization
import seaborn as sns

In [3]:
# Load the chat file
with open('data/_chat.txt', encoding='utf-8') as file:
    lines = file.readlines()

In [4]:
# Initialize lists to store data
dates = []
times = []
authors = []
messages = []

# Regular expression to match the date, time, author, and message
pattern = r'(\d+/\d+/\d+), (\d+:\d+) - (.*?): (.*)'

In [ ]:
for line in lines:
    match = re.match(pattern, line)
    if match:
        dates.append(match.group(1))
        times.append(match.group(2))
        authors.append(match.group(3))
        messages.append(match.group(4))
    else:  # Continuation of a message
        messages[-1] += line